In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import yfinance as yf
import ta  # Technical Analysis library for financial data
import joblib

# Load historical forex data for the last 30 days at 1-minute intervals
data = yf.download('EURUSD=X', start='2023-08-20', end='2023-09-19', interval='1m')
data.reset_index(inplace=True)

# Display the data
print(data.head())

# Feature Engineering
# Calculate moving averages
data['MA_10'] = data['Close'].rolling(window=10).mean()
data['MA_50'] = data['Close'].rolling(window=50).mean()
data['MA_200'] = data['Close'].rolling(window=200).mean()

# Calculate RSI
data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()

# Calculate MACD
data['MACD'] = ta.trend.MACD(data['Close']).macd()

# Calculate Bollinger Bands
bollinger = ta.volatility.BollingerBands(data['Close'])
data['BB_High'] = bollinger.bollinger_hband()
data['BB_Low'] = bollinger.bollinger_lband()

# Calculate Daily Returns
data['Daily_Return'] = data['Close'].pct_change()

# Drop rows with NaN values (due to moving averages and indicators)
data.dropna(inplace=True)

# Feature and target variables
X = data[['MA_10', 'MA_50', 'MA_200', 'RSI', 'MACD', 'BB_High', 'BB_Low', 'Daily_Return']]
y = data['Close']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

# Initialize the model (Random Forest as an example)
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted Prices', color='orange')
plt.title('Forex Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Save the model (Optional)
joblib.dump(model, 'forex_model.pkl')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['EURUSD=X']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1m 2023-08-20 -> 2023-09-19) (Yahoo error = "1m data not available for startTime=1692486000 and endTime=1695078000. Only 7 days worth of 1m granularity data are allowed to be fetched per request.")')


Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume]
Index: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.